In [1]:
import sys
import os
import re
import pandas as pd
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers, Sequential, Input
import datetime


In [2]:
pip install ann_visualizer

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
load_ext tensorboard

# Fetching Datasets

In [4]:
#@title Fetching MHealth
# MHEALTH http://archive.ics.uci.edu/ml/machine-learning-databases/00319/MHEALTHDATASET.zip

In [5]:
unzip "C:/Users/dixit/Desktop/uci-har dataset/UCI HAR Dataset"

SyntaxError: invalid syntax (106886225.py, line 1)

In [ ]:
  #@title Packages
!apt install poppler-utils

## Dataset Description

In [ ]:
#@title MHealth Dataset Readme
! cat MHEALTHDATASET/README.txt

In [ ]:
#@title -

!pdftotext PAMAP2_Dataset/readme.pdf  

## Importing Dataset

In [ ]:
#@title Inspecting MHealth log File
!head -n 10 MHEALTHDATASET/mHealth_subject1.log

In [ ]:
#@title MHealth Files List
for i in os.listdir('MHEALTHDATASET/'):
  if not re.search('README', i):
    print(i)

In [ ]:
#@title Converting one subject to DataFrame
mHealth = pd.read_csv('MHEALTHDATASET/mHealth_subject3.log', header=None, sep='\t').loc[:, [5,6,7,8,9,10,14,15,16,17,18,19, 23]].rename(columns={5:"alX", 6:"alY", 7:"alZ", 8:"glX", 9:"glY", 10:"glZ", 14:"arX",15:"arY", 16:"arZ", 17:"grX", 18:"grY", 19:"grZ", 23:"Output"})
mHealth.head(5)

In [ ]:
sns.pairplot(mHealth)

# EDA

In [ ]:
#@title Describe MHealth Dataset One Subject
mHealth.describe().T

In [ ]:
#@title Describe MHealth for all
all = mHealth[mHealth["Output"] == 14].copy()

for i in os.listdir('MHEALTHDATASET/'):
  if not re.search('README', i):
    all = all.append(pd.read_csv('MHEALTHDATASET/'+i, header=None, sep='\t').loc[:, [5,6,7,8,9,10,14,15,16,17,18,19, 23]].rename(columns={5:"alX", 6:"alY", 7:"alZ", 8:"glX", 9:"glY", 10:"glZ", 14:"arX",15:"arY", 16:"arZ", 17:"grX", 18:"grY", 19:"grZ", 23:"Output"}))


description = all.describe()

description.T

In [ ]:
description["alX"]["max"]

# MHealth Dataset

In [ ]:
#@title Size of dataset for various subjects
for i in os.listdir('MHEALTHDATASET/'):
  if not re.search('README', i):
    print(pd.read_csv('MHEALTHDATASET/'+i, header=None, sep='\t').loc[:, [5,6,7,8,9,10,14,15,16,17,18,19, 23]].rename(columns={5:"alX", 6:"alY", 7:"alZ", 8:"glX", 9:"glY", 10:"glZ", 14:"arX",15:"arY", 16:"arZ", 17:"grX", 18:"grY", 19:"grZ", 23:"Output"}).shape)


In [ ]:
all.isnull().values.any()

In [ ]:
#@title Importing and preprocessing dataset
SAMPLES_PER_ACTIVITY=50  #@param { "type": "integer" }

Activities = [
    "Nothing",
    "Standing",
    "Sitting and Relaxing",
    "Lying Down",
    "Walking",
    "Climbing Stairs",
    "Waist bends forward",
    "Frontal Elevation of Arms",
    "Knees Bending (Crouching)",
    "Cycling",
    "Jogging",
    "Running",
    "Jump Front & Back"
    ]

Activities_subset = [4, 1, 5, 9, 10, 11, 8]
NUM_ACTIVITY=len(Activities)
X = []
Y = []

FEATURES=["alX", "alY", "alZ", "glX", "glY", "glZ", "arX", "arY", "arZ", "grX", "grY", "grZ"]
ONE_HOT_ENCODED_ACTIVITY = np.eye(NUM_ACTIVITY-1)
for i in os.listdir('MHEALTHDATASET/'):
  if not re.search('README', i):
    temp = pd.read_csv('MHEALTHDATASET/'+i, header=None, sep='\t').loc[:, [5,6,7,8,9,10,14,15,16,17,18,19, 23]].rename(columns={5:"alX", 6:"alY", 7:"alZ", 8:"glX", 9:"glY", 10:"glZ", 14:"arX",15:"arY", 16:"arZ", 17:"grX", 18:"grY", 19:"grZ", 23:"Output"})
    for ji,j in enumerate(Activities_subset):

        if j not in Activities_subset: continue
        output = ONE_HOT_ENCODED_ACTIVITY[ji]

        
        df =np.array(temp[temp["Output"] == j].copy())
        num_recordings = int(df.shape[0]/SAMPLES_PER_ACTIVITY)
        print(f"\tThere are {num_recordings} recordings of the {Activities[j]} activity.")
        for k in range(num_recordings):
          tensor = []
          for l in range(SAMPLES_PER_ACTIVITY):
            index = k * SAMPLES_PER_ACTIVITY + l

            tensori = []
            for m,feature in enumerate(FEATURES):
              tensori.append((df[index,m] + description[feature]["mean"]) / (description[feature]["max"] - description[feature]["min"]))
            
            tensor.append(tensori)
          X.append(tensor)
          Y.append(output)

X = np.array(X)
Y = np.array(Y)


print("Data set parsing and preparation complete.")



In [ ]:
X.min()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

In [ ]:
input_shape=X.shape[1:]

X.shape

In [ ]:
import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, TimeDistributed
from keras.layers import Conv3D,Conv2D,Conv1D,MaxPooling1D, MaxPooling2D,TimeDistributed,LSTM,ConvLSTM2D, GRU
from keras.utils import np_utils

from sklearn.metrics import classification_report

# Models

## Convnet

In [ ]:
model = Sequential()
model.add(Conv1D(32, 3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(12, activation='softmax'))
model.summary()

In [ ]:

model.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

In [ ]:
BATCH_SIZE = 64

EPOCHS=40

In [ ]:
history = model.fit(X_train,
                      Y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      shuffle=True,
                      validation_split=0.3,
                      verbose=1)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], 'r', label='Accuracy of training data')
plt.plot(history.history['val_accuracy'], 'b', label='Accuracy of validation data')
plt.plot(history.history['loss'], 'r--', label='Loss of training data')
plt.plot(history.history['val_loss'], 'b--', label='Loss of validation data')
plt.title('Model Accuracy and Loss')
plt.ylabel('Accuracy and Loss')
plt.xlabel('Training Epoch')
plt.ylim(0)
plt.legend()
plt.show()

# Print confusion matrix for training data




In [ ]:
predictions = model.predict(X_test)

cm = tf.math.confusion_matrix(labels=tf.argmax(Y_test,1), predictions=tf.argmax(predictions,1))
axis_labels = [Activities[i] for i in Activities_subset]
figure = plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True,cmap=plt.cm.Blues,xticklabels=axis_labels, yticklabels=axis_labels, fmt='g')
plt.ylim(len(cm)-0, 0)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
pd.DataFrame(classification_report(Y_test.argmax(axis=1), predictions.argmax(axis=1), output_dict=True)).rename(columns=dict(zip([str(i) for i in range(0,7)], list([Activities[Activities_subset[i]] for i in range(0,7)])))).T

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model)

## CNN-LSTM

In [ ]:
model = Sequential()
model.add(Conv1D(32, 3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(128))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(12, activation='softmax'))
model.summary()

In [ ]:

model.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,
                      Y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,shuffle=True,
                      validation_split=0.3,
                      verbose=1)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], 'r', label='Accuracy of training data')
plt.plot(history.history['val_accuracy'], 'b', label='Accuracy of validation data')
plt.plot(history.history['loss'], 'r--', label='Loss of training data')
plt.plot(history.history['val_loss'], 'b--', label='Loss of validation data')
plt.title('Model Accuracy and Loss')
plt.ylabel('Accuracy and Loss')
plt.xlabel('Training Epoch')
plt.ylim(0)
plt.legend()
plt.show()

# Print confusion matrix for training data




In [ ]:
predictions = model.predict(X_test)

cm = tf.math.confusion_matrix(labels=tf.argmax(Y_test,1), predictions=tf.argmax(predictions,1))
axis_labels = [Activities[i] for i in Activities_subset]
figure = plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True,cmap=plt.cm.Blues,xticklabels=axis_labels, yticklabels=axis_labels, fmt='g')
plt.ylim(len(cm)-0, 0)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
pd.DataFrame(classification_report(Y_test.argmax(axis=1), predictions.argmax(axis=1), output_dict=True)).rename(columns=dict(zip([str(i) for i in range(0,7)], list([Activities[Activities_subset[i]] for i in range(0,7)])))).T

In [ ]:
plot_model(model)

## LSTM-CNN

In [ ]:
model = Sequential()

model.add(LSTM(128, input_shape=input_shape))
model.add(Reshape((-1,1)))
model.add(Conv1D(32, 3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(12, activation='softmax'))
model.summary()

In [ ]:

model.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,
                      Y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,shuffle=True,
                      validation_split=0.3,
                      verbose=1)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], 'r', label='Accuracy of training data')
plt.plot(history.history['val_accuracy'], 'b', label='Accuracy of validation data')
plt.plot(history.history['loss'], 'r--', label='Loss of training data')
plt.plot(history.history['val_loss'], 'b--', label='Loss of validation data')
plt.title('Model Accuracy and Loss')
plt.ylabel('Accuracy and Loss')
plt.xlabel('Training Epoch')
plt.ylim(0)
plt.legend()
plt.show()

# Print confusion matrix for training data




In [ ]:
predictions = model.predict(X_test)

cm = tf.math.confusion_matrix(labels=tf.argmax(Y_test,1), predictions=tf.argmax(predictions,1))
axis_labels = [Activities[i] for i in Activities_subset]
figure = plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True,cmap=plt.cm.Blues,xticklabels=axis_labels, yticklabels=axis_labels, fmt='g')
plt.ylim(len(cm)-0, 0)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
pd.DataFrame(classification_report(Y_test.argmax(axis=1), predictions.argmax(axis=1), output_dict=True)).rename(columns=dict(zip([str(i) for i in range(0,7)], list([Activities[Activities_subset[i]] for i in range(0,7)])))).T

In [ ]:
plot_model(model)

## LSTM

In [ ]:
model = Sequential()

model.add(LSTM(128, input_shape=input_shape))

model.add(Dense(12, activation='softmax'))
model.summary()

In [ ]:

model.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])


In [ ]:
history = model.fit(X_train,
                      Y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,shuffle=True,
                      validation_split=0.3,
                      verbose=1)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], 'r', label='Accuracy of training data')
plt.plot(history.history['val_accuracy'], 'b', label='Accuracy of validation data')
plt.plot(history.history['loss'], 'r--', label='Loss of training data')
plt.plot(history.history['val_loss'], 'b--', label='Loss of validation data')
plt.title('Model Accuracy and Loss')
plt.ylabel('Accuracy and Loss')
plt.xlabel('Training Epoch')
plt.ylim(0)
plt.legend()
plt.show()

# Print confusion matrix for training data




In [ ]:
predictions = model.predict(X_test)

cm = tf.math.confusion_matrix(labels=tf.argmax(Y_test,1), predictions=tf.argmax(predictions,1))
axis_labels = [Activities[i] for i in Activities_subset]
figure = plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True,cmap=plt.cm.Blues,xticklabels=axis_labels, yticklabels=axis_labels, fmt='g')
plt.ylim(len(cm)-0, 0)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
pd.DataFrame(classification_report(Y_test.argmax(axis=1), predictions.argmax(axis=1), output_dict=True)).rename(columns=dict(zip([str(i) for i in range(0,7)], list([Activities[Activities_subset[i]] for i in range(0,7)])))).T

In [ ]:
plot_model(model)

## CNN-GRU

In [ ]:
model = Sequential()
model.add(Conv1D(32, 3, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(TimeDistributed(Flatten()))
model.add(GRU(128))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(12, activation='softmax'))
model.summary()

In [ ]:

model.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,
                      Y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,shuffle=True,
                      validation_split=0.3,
                      verbose=1)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], 'r', label='Accuracy of training data')
plt.plot(history.history['val_accuracy'], 'b', label='Accuracy of validation data')
plt.plot(history.history['loss'], 'r--', label='Loss of training data')
plt.plot(history.history['val_loss'], 'b--', label='Loss of validation data')
plt.title('Model Accuracy and Loss')
plt.ylabel('Accuracy and Loss')
plt.xlabel('Training Epoch')
plt.ylim(0)
plt.legend()
plt.show()

# Print confusion matrix for training data




In [ ]:
predictions = model.predict(X_test)

cm = tf.math.confusion_matrix(labels=tf.argmax(Y_test,1), predictions=tf.argmax(predictions,1))
axis_labels = [Activities[i] for i in Activities_subset]
figure = plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True,cmap=plt.cm.Blues,xticklabels=axis_labels, yticklabels=axis_labels, fmt='g')
plt.ylim(len(cm)-0, 0)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
pd.DataFrame(classification_report(Y_test.argmax(axis=1), predictions.argmax(axis=1), output_dict=True)).rename(columns=dict(zip([str(i) for i in range(0,7)], list([Activities[Activities_subset[i]] for i in range(0,7)])))).T

In [ ]:
plot_model(model)

## GRU-CNN

In [ ]:
model = Sequential()
model.add(GRU(128, input_shape=input_shape))

model.add(Reshape((-1,1)))
model.add(Conv1D(32, 3, activation='relu'))

model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(12, activation='softmax'))
model.summary()

In [ ]:

model.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,
                      Y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,shuffle=True,
                      validation_split=0.3,
                      verbose=1)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], 'r', label='Accuracy of training data')
plt.plot(history.history['val_accuracy'], 'b', label='Accuracy of validation data')
plt.plot(history.history['loss'], 'r--', label='Loss of training data')
plt.plot(history.history['val_loss'], 'b--', label='Loss of validation data')
plt.title('Model Accuracy and Loss')
plt.ylabel('Accuracy and Loss')
plt.xlabel('Training Epoch')
plt.ylim(0)
plt.legend()
plt.show()

# Print confusion matrix for training data




In [ ]:
predictions = model.predict(X_test)

cm = tf.math.confusion_matrix(labels=tf.argmax(Y_test,1), predictions=tf.argmax(predictions,1))
axis_labels = [Activities[i] for i in Activities_subset]
figure = plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True,cmap=plt.cm.Blues,xticklabels=axis_labels, yticklabels=axis_labels, fmt='g')
plt.ylim(len(cm)-0, 0)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
pd.DataFrame(classification_report(Y_test.argmax(axis=1), predictions.argmax(axis=1), output_dict=True)).rename(columns=dict(zip([str(i) for i in range(0,7)], list([Activities[Activities_subset[i]] for i in range(0,7)])))).T

In [ ]:
plot_model(model)

## GRU

In [ ]:
model = Sequential()
model.add(GRU(128, input_shape=input_shape))
model.add(Dense(12, activation='softmax'))
model.summary()

In [ ]:

model.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,
                      Y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,shuffle=True,
                      validation_split=0.3,
                      verbose=1)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], 'r', label='Accuracy of training data')
plt.plot(history.history['val_accuracy'], 'b', label='Accuracy of validation data')
plt.plot(history.history['loss'], 'r--', label='Loss of training data')
plt.plot(history.history['val_loss'], 'b--', label='Loss of validation data')
plt.title('Model Accuracy and Loss')
plt.ylabel('Accuracy and Loss')
plt.xlabel('Training Epoch')
plt.ylim(0)
plt.legend()
plt.show()

# Print confusion matrix for training data




In [ ]:
predictions = model.predict(X_test)

cm = tf.math.confusion_matrix(labels=tf.argmax(Y_test,1), predictions=tf.argmax(predictions,1))
axis_labels = [Activities[i] for i in Activities_subset]
figure = plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True,cmap=plt.cm.Blues,xticklabels=axis_labels, yticklabels=axis_labels, fmt='g')
plt.ylim(len(cm)-0, 0)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
pd.DataFrame(classification_report(Y_test.argmax(axis=1), predictions.argmax(axis=1), output_dict=True)).rename(columns=dict(zip([str(i) for i in range(0,7)], list([Activities[Activities_subset[i]] for i in range(0,7)])))).T

In [ ]:
plot_model(model) 